In [ ]:
import datetime
import knmi
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from knmi.metadata import Station

STATIONS = knmi.stations

def get_time():
    #TODO: hourly data isn't up to date enough, go for exactly 1 day ago instead.
    now = datetime.datetime.now()
    yesterday = now - datetime.timedelta(days=1)
    return yesterday.strftime("%Y%m%d%H") # formatted as YYYYMMDDHH

def get_location(input_city):
    coordinates = _get_input_coordinates(input_city)
    nearest_station, min_distance = _get_nearest_station(coordinates)
    return nearest_station, min_distance

def _get_input_coordinates(city_name):
    geolocator = Nominatim(user_agent="knmi_city_lookup")
    location = geolocator.geocode(city_name, featuretype="city")
    if not location:
        raise ValueError(f"Could not find coordinates for city: {city_name}")
    location = geolocator.geocode(city_name, country_codes="nl", featuretype="city")
    if not location:
        raise ValueError(f"The city of {city_name} is not in the Netherlands.")
    return location.latitude, location.longitude


def _get_nearest_station(city_coords):
    min_distance = float("inf")
    nearest_station = None

    for station in STATIONS.values():
        station_coords = (station.latitude, station.longitude)
        distance = geodesic(city_coords, station_coords).kilometers
        if distance < min_distance:
            min_distance = distance
            nearest_station = station

    return nearest_station, min_distance

In [ ]:


def get_pressure(station: Station, time: datetime.datetime):
    time = time.strftime("%Y%m%d%H")
    pressure_df = knmi.get_hour_data_dataframe(
        stations=[station.number],
        start=time,
        end=time,
        variables=["P"],
    )
    return pressure_df["P"].values[0]


In [ ]:
input_city = "Utrecht"
# current_time = get_time()

now = datetime.datetime.now()
yesterday = now - datetime.timedelta(days=1)
lastweek = now - datetime.timedelta(days=7)

try:
    nearest_station, distance = get_location(input_city)
    print(f"Nearest weather station to {input_city}: {nearest_station.name} ({nearest_station.number})")
    print(f"Distance: {distance:.2f} km")

    pressure = get_pressure(nearest_station, lastweek)
    print(f"Pressure on {lastweek.strftime('%Y-%m-%d %H:00')}: {pressure} hPa")
except ValueError as e:
    print(e)


In [ ]:
type(now)

In [ ]:
_get_input_coordinates("test")